# Text Analysis on ottawashooting data

In [26]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [28]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,0
50,524922499466022913,18999969,Passersby working on shooting victim. From the...,Wed Oct 22 13:57:31 +0000 2014,0
65,524923148576518144,81122301,Shooting at the war memorial in Ottawa. http:/...,Wed Oct 22 14:00:05 +0000 2014,0
87,524923403183333376,150425432,After today we will officially be done shootin...,Wed Oct 22 14:01:06 +0000 2014,0
106,524924660480487424,15996661,KEY: Nothing in Wilson's story in new autopsy ...,Wed Oct 22 14:06:06 +0000 2014,0
...,...,...,...,...,...
11328,525068915068923904,17820367,***EXCLUSIVE*** Michael Zahaf-Bibeau caught on...,Wed Oct 22 23:39:19 +0000 2014,1
11331,525070933556350977,351120282,"And this is Cpl. Nathan Cirillo, the brave #Ca...",Wed Oct 22 23:47:20 +0000 2014,1
11350,525071376084791297,25235198,"Rest in Peace, Cpl. Nathan Cirillo. Killed tod...",Wed Oct 22 23:49:06 +0000 2014,1
11363,525071638464049152,19038934,Watch live: PM Stephen Harper expected to spea...,Wed Oct 22 23:50:08 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [29]:
def textprocessing(text):
    text = text.lower()

    text = " ".join([word for word in text.split() if 'http' not in word
                                and not word.startswith('@')])
                                #and word != 'RT'])
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #how to remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))
          
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    text = [stem.stem(i) for i in text]
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    text = " ".join(text)
    
    return text

In [30]:
print("original text: ", df.reply_tweet.iloc[20])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[20]))

original text:  @StephBerthomet @en_passant_la Sauf que ça c'est retrouver partout bien avant d'être confirmé. Bien triste nouvelle au final.
cleaned text:  sauf que ça cest retrouv partout bien avant dêtre confirmé bien trist nouvel au final


### applied on original data (**df**)

In [31]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [32]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw
0,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524922392582586368,2433907530,.@dominiquehardy @CBCOttawa related to St-Jean ?,Wed Oct 22 13:57:05 +0000 2014,0,ottawa polic confirm shoot war memori minut ag...,dominiquehardi relat stjean
1,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524922589861658624,15816550,@petitmetroide @CBCOttawa Il est tôt pour une ...,Wed Oct 22 13:57:52 +0000 2014,0,ottawa polic confirm shoot war memori minut ag...,il est tôt pour une quelconqu hypothès
2,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524922681595269120,1206639234,Prudence avant d'en savoir plus “@CBCOttawa: O...,Wed Oct 22 13:58:14 +0000 2014,0,ottawa polic confirm shoot war memori minut ag...,prudenc avant den savoir plu cbcottawa ottawa ...
3,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524923071090917376,347531577,@StephBerthomet Ça va être rapporté partout sa...,Wed Oct 22 13:59:47 +0000 2014,0,ottawa polic confirm shoot war memori minut ag...,ça va être rapporté partout san validationsur ...
4,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524923096911052802,96146974,“@CBCOttawa: Ottawa police are confirming a sh...,Wed Oct 22 13:59:53 +0000 2014,0,ottawa polic confirm shoot war memori minut ag...,cbcottawa ottawa polic confirm shoot war memor...


### applied on source tweet data (**src_tw_df**) 

In [33]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)

## Sentiment Analysis

In [34]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [35]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [36]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [37]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw,src_sentiment,reply_sentiment
0,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524922392582586368,2433907530,.@dominiquehardy @CBCOttawa related to St-Jean ?,Wed Oct 22 13:57:05 +0000 2014,0,ottawa polic confirm shoot war memori minut ag...,dominiquehardi relat stjean,Negative,Neutral


### applied on source tweet data only 

In [38]:
### source tweet data only 
src_tw_df['src_sentiment']=  src_tw_df['cleaned_src_tw'].apply(get_sentiment)

### Visualization

In [ ]:
print(df[df.label ==1]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 1])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Rumour',fontsize=16)
plt.title('Reply tweets show more positive attitude towards rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/ottawashooting/senti_reply_rumour.pdf',dpi=300)

In [ ]:
print(df[df.label ==0]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 0])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Non-Rumour',fontsize=16)
plt.title('Reply tweets show more neutrual attitude towards non-rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/ottawashooting/senti_reply_nonrumour.pdf',dpi=300)

In [ ]:
print(src_tw_df['src_sentiment'].value_counts())
sns.countplot(x='src_sentiment',data =src_tw_df,hue='label')

plt.suptitle('Sentiment analysis on Source Tweets',fontsize=16)
plt.title('Rumour source tweets show more neutral attitude.',fontsize=12,color='grey')
plt.savefig('graph/ottawashooting/senti_source.pdf',dpi=300)

### save data to csv 

In [ ]:
# df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ottawashooting/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ottawashooting/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ottawashooting/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [39]:
import text2emotion as te

In [40]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [46]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['reply_emotion']=emo

In [47]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-df.csv',index=False)

In [48]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['src_emotion']=emo1

In [49]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-src.csv',index=False)

In [45]:
src_tw_df_1 = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-src.csv', encoding = 'utf-8', header =0)